#### QandA implementation using haystack
---


In [2]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [16]:
# Connect to Elasticsearch

#from haystack.database.elasticsearch import ElasticsearchDocumentStore
#document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

# In-Memory Document Store
from haystack.database.memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [17]:
doc_dir = '/home/krishna/projects/haystack/docs/'

# Now, let's write the docs to our DB.
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
write_documents_to_db(document_store=document_store, document_dir=doc_dir, \
                      only_empty_db=True, split_paragraphs=True)

05/01/2020 18:16:22 - INFO - haystack.indexing.io -   Wrote 500 docs to DB


In [18]:
## Using BM25
##from haystack.retriever.elasticsearch import ElasticsearchRetriever
##retriever = ElasticsearchRetriever(document_store=document_store)

# An in-memory TfidfRetriever based on Pandas dataframes

from haystack.retriever.tfidf import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

05/01/2020 18:16:54 - INFO - haystack.retriever.tfidf -   Found 500 candidate paragraphs from 500 docs in DB


In [26]:
### Reader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)
#reader = TransformersReader(model="deepset/roberta-base-squad2",tokenizer="deepset/roberta-base-squad2-covid", use_gpu=-1)

05/01/2020 18:20:10 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
05/01/2020 18:20:10 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
05/01/2020 18:20:15 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
05/01/2020 18:20:22 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
05/01/2020 18:20:24 - INFO - filelock -   Lock 139774506852192 acquired on /home/krishna/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


05/01/2020 18:20:27 - INFO - filelock -   Lock 139774506852192 released on /home/krishna/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


05/01/2020 18:20:28 - INFO - filelock -   Lock 139774506852192 acquired on /home/krishna/.cache/torch/transformers/b901c69e8e7da4a24c635ad81d016d274f174261f4f5c144e43f4b00e242c3b0.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


05/01/2020 18:20:31 - INFO - filelock -   Lock 139774506852192 released on /home/krishna/.cache/torch/transformers/b901c69e8e7da4a24c635ad81d016d274f174261f4f5c144e43f4b00e242c3b0.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


05/01/2020 18:20:33 - INFO - filelock -   Lock 139774506852192 acquired on /home/krishna/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


05/01/2020 18:20:34 - INFO - filelock -   Lock 139774506852192 released on /home/krishna/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


05/01/2020 18:20:35 - INFO - filelock -   Lock 139774506938208 acquired on /home/krishna/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.faa301f950f21d5033722a5b7fa9914cd6f649bb9db3d18b2976d5c6d55aed3d.lock


05/01/2020 18:20:36 - INFO - filelock -   Lock 139774506938208 released on /home/krishna/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.faa301f950f21d5033722a5b7fa9914cd6f649bb9db3d18b2976d5c6d55aed3d.lock
05/01/2020 18:20:37 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None


In [27]:
finder = Finder(reader, retriever)

In [28]:
prediction = finder.get_answers(question="what is Covid-19?", top_k_retriever=1, top_k_reader=5)

05/01/2020 18:20:45 - INFO - haystack.retriever.tfidf -   Identified 1 candidates via retriever:
  paragraph_id                       document_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

05/01/2020 18:20:45 - INFO - haystack.finder -   Reader is looking for detailed answer in 10246 chars ...


OrderedDict([(232, array([0.18665962])), (233, array([0.17970997])), (487, array([0.17916202])), (456, array([0.17714134])), (12, array([0.17428591])), (136, array([0.1703749])), (362, array([0.1698908])), (207, array([0.16347776])), (0, array([0.16253012])), (225, array([0.16066024])), (308, array([0.15755865])), (84, array([0.15376142])), (416, array([0.15289455])), (141, array([0.15004801])), (124, array([0.1478327])), (98, array([0.14662645])), (412, array([0.14573767])), (423, array([0.14473539])), (274, array([0.14458955])), (131, array([0.13704985])), (438, array([0.13575856])), (59, array([0.13542492])), (295, array([0.13504204])), (65, array([0.13442388])), (323, array([0.13375939])), (277, array([0.1330447])), (271, array([0.13023316])), (180, array([0.12896607])), (215, array([0.1289261])), (305, array([0.12851698])), (255, array([0.12843555])), (475, array([0.12680846])), (121, array([0.12453702])), (250, array([0.12390663])), (46, array([0.12365355])), (434, array([0.12310

In [31]:
print_answers(prediction, details="medium")

[   {   'answer': 'a global pandemic',
        'context': 'pa, S Soubeyrand, , 2020; Mathematical predictions for '
                   'covid-19 as a global pandemic. medRxiv preprint, V A '
                   'Okhuese, , 2020; The three phases of COVID'},
    {   'answer': 'CC-BY-NC-ND 4.0 International license',
        'context': 'e 2 and is predicted to have above 100 for next week. . '
                   'CC-BY-NC-ND 4.0 International license It is made available '
                   'under a author/funder, who has gran'},
    {   'answer': 'pandemic',
        'context': ' reported from India. Coming weeks are crucial for '
                   'containing COVID-19 pandemic. It is predicted to cross '
                   'cumulative cases of 3,000within Apr 3. autho'}]
